# Calculating Churn Rates

Four months into launching Codeflix, management asks you to look into subscription churn rates. It’s early on in the business and people are excited to know how the company is doing.

The marketing department is particularly interested in how the churn compares between two segments of users. They provide you with a dataset containing subscription data for users who were acquired through two distinct channels.

The dataset provided to you contains one SQL table, subscriptions. Within the table, there are 4 columns:

- id - the subscription id
- subscription_start - the start date of the subscription
- subscription_end - the end date of the subscription
- segment - this identifies which segment the subscription owner belongs to

Codeflix requires a minimum subscription length of 31 days, so a user can never start and end their subscription in the same month.

In [ ]:
/* Get familiar with the data */
/* Take a look at the first 100 rows of data in the subscriptions table. */
SELECT *
FROM subscriptions
LIMIT 50;

/* Determine the range of months of data provided. */
SELECT MIN(subscription_start), MAX(subscription_start)
FROM subscriptions;

/* You’ll be calculating the churn rate for both segments (87 and 30) 
over the first 3 months of 2017 (you can’t calculate it for December, 
since there are no subscription_end values yet). */
WITH months AS
(SELECT 
	'2017-01-01' as first_day,
	'2017-01-31' as last_day
UNION
SELECT
	'2017-02-01' as first_day,
	'2017-02-28' as last_day
UNION
SELECT
	'2017-03-01' as first_day,
	'2017-03-31' as last_day
),
/* Create a temporary table, cross_join, from subscriptions and your months.*/
cross_join AS
(SELECT *
FROM subscriptions
CROSS JOIN months
),
/* Create a temporary table, status, from the cross_join table you created.*/
status AS
(SELECT id, first_day as 'month',
	CASE
		WHEN ((segment = 87)
			AND (subscription_start < first_day
          AND (subscription_end > first_day
              OR subscription_end IS NULL)
           )) THEN 1
 		ELSE 0
 END as 'is_active_87',
 CASE
		WHEN ((segment = 30)
			AND (subscription_start < first_day
          AND (subscription_end > first_day
              OR subscription_end IS NULL)
           )) THEN 1
 		ELSE 0
 END as 'is_active_30',
 CASE
		WHEN ((segment = 87)
			AND (subscription_start < first_day
          AND (subscription_end BETWEEN first_day
              AND last_day)
           )) THEN 1
 		ELSE 0
 END as 'is_canceled_87',
 CASE
		WHEN ((segment = 30)
			AND (subscription_start < first_day
          AND (subscription_end BETWEEN first_day
              AND last_day)
           )) THEN 1
 		ELSE 0
 END as 'is_canceled_30'
FROM cross_join),
/* 
Create a status_aggregate temporary table that is a SUM of the active and canceled subscriptions
for each segment, for each month.*/
status_aggregate AS
(SELECT month,
 		SUM(is_active_87) as 'sum_active_87', SUM(is_active_30) as 'sum_active_30',
		SUM(is_canceled_87) as 'sum_canceled_87', SUM(is_canceled_30) as 'sum_canceled_30'
FROM status
GROUP BY month
)
/* Calculate the churn rates for the two segments over the three month period. */
SELECT month, 
		1.0 * sum_canceled_87 / sum_active_87 as '87_churn_rate',
        1.0 * sum_canceled_30 / sum_active_30 as '30_churn_rate'
FROM status_aggregate;